# MSA 2025 Phase 2 - Part 2: Minimum Temperature Prediction from Weather Data

## 1. Load and check the dataset

In [142]:
#load useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
#import the data
df = pd.read_csv("Summary of Weather.csv", encoding='latin1')
df.shape[0] # number of rows in the dataframe

C:\Users\GGPC\AppData\Local\Temp\ipykernel_26472\482264606.py:2: DtypeWarning: Columns (7,8,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Summary of Weather.csv", encoding='latin1')


119040

In [144]:
missing_data = df.isnull().sum().to_frame()

# Rename column holding the sums
missing_data = missing_data.rename(columns={0:'Empty Cells'})

# Print the results
print(missing_data)

             Empty Cells
STA                    0
Date                   0
Precip                 0
WindGustSpd       118508
MaxTemp                0
MinTemp                0
MeanTemp               0
Snowfall            1163
PoorWeather        84803
YR                     0
MO                     0
DA                     0
PRCP                1932
DR                118507
SPD               118508
MAX                  474
MIN                  468
MEA                  498
SNF                 1163
SND               113477
FT                119040
FB                119040
FTI               119040
ITH               119040
PGT               118515
TSHDSBRSGF         84803
SD3               119040
RHX               119040
RHN               119040
RVG               119040
WTE               119040


In [145]:
df = df.convert_dtypes()
df.dtypes

STA                     Int64
Date           string[python]
Precip         string[python]
WindGustSpd           Float64
MaxTemp               Float64
MinTemp               Float64
MeanTemp              Float64
Snowfall               object
PoorWeather            object
YR                      Int64
MO                      Int64
DA                      Int64
PRCP           string[python]
DR                      Int64
SPD                     Int64
MAX                     Int64
MIN                     Int64
MEA                     Int64
SNF                    object
SND                     Int64
FT                      Int64
FB                      Int64
FTI                     Int64
ITH                     Int64
PGT                   Float64
TSHDSBRSGF             object
SD3                     Int64
RHX                     Int64
RHN                     Int64
RVG                     Int64
WTE                     Int64
dtype: object

## 2. Clean the dataset, drop variables

## 3. Select the feature(s) for the model, and explain the reason

In [20]:
# write your answer here

## 4. Split the dataset (80% for training, 20% for testing)

## 5. Choose the algorithm, train and test the model

## 6. Evaluate and Visualize model Performance

## 7. What is the R² value and RMSE of the model? How these values indicate the model performance?

In [21]:
# write your answer here

## 8. Discuss the model performance based on the plot

In [22]:
# write your answer here